# Gradient-based attacks

Adapted from an adversarial ML tutorial by Kolter and Madry first delivered at NeurIPS 2018. Don't look it up before you are done, unless you want to spoil the solutions :)

In [ ]:
### PREAMBLE
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

Let's consider the adversarial example generation problem
\begin{equation}
\DeclareMathOperator*{\maximize}{maximize}
\maximize_{\|\delta\| \leq \epsilon} \ell(h_\theta(x), y)
\end{equation}
where $h_\theta(x)$ now represents a neural network. Let us assume that the loss function is the cross entropy loss
\begin{equation}
\ell(h_\theta(x), y) = \log \left ( \sum_{j=1}^k \exp(h_\theta(x)_j) \right ) - h_\theta(x)_y.
\end{equation}

The loss surface of standard neural networks is very "irregular".  While we won't go into too much detail on this point, the underlying issue here is that because neural networks have much more modeling power than linear models, they have the ability to have much bumpier function surface. For example, consider the hypothesis for a simple (randomly trained) model below, compared to a simple linear function.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(0)
model = nn.Sequential(nn.Linear(1,100), nn.ReLU(), 
                      nn.Linear(100,100), nn.ReLU(), 
                      nn.Linear(100,100), nn.ReLU(), 
                      nn.Linear(100,1))
opt = optim.SGD(model.parameters(),lr=1e-2)
for _ in range(100):
    loss = nn.MSELoss()(model(torch.randn(100,1)), torch.randn(100,1))
    opt.zero_grad()
    loss.backward()
    opt.step()
                      
plt.plot(np.arange(-3,3,0.01), model(torch.arange(-3,3,0.01)[:,None]).detach().numpy())
plt.xlabel("Input")
plt.ylabel("Output")

These loss surfaces lead to two main challenges. First, in the high dimensional setting that we will typically consider with deep networks, there is a high likelihood that at almost any point in input space there will be _some_ direction along the loss surface that is very steep, i.e., which causes the loss to either increase or decrease substantially.  This is actually the mathematical equivalent of what we saw in the introductory chapter: that we can move a small distance in input space (i.e., have a small perturbation), which leads to a big increase in loss.  Put another way, _neural networks, by the nature of their loss surfaces, are especially prone to adversarial examples_.

It is _not_ easy to solve (or easily upper bound) the optimization problem problem over our perturbation.  This is because, as can be seen in the figure above, the cost surface for neural networks (here considering it over the _input_ space, not parameter space) is not convex, and is especially prone to local optima.  We can see this even in the simple example above, where there are local optimal when either trying to maximize or minimnize the function above, and the initial gradient at a given point may or may not point in the direction of the actual maxima.  This second point may be somewhat less of an issue if our goal is just to _construct_ an adversarial example against a standard deep network (after all, as the first point illustrated, there are many directions of high cost increase, and just following the gradient typically leads to _an_ adversarial example, even if it is not the _optimal_ adversarial example.

### Strategies

So how _do_ we go about (approximately) solving optimization problem
\begin{equation}
\maximize_{\|\delta\| \leq \epsilon} \ell(h_\theta(x), y)
\end{equation}
in the case of $h_\theta$ being a neural network? There are three main strategies for doing so, which relate to _lower bounds, exact solutions, and upper bounds_ on this optimization objective.  In more detail, we have the following options:

1. We can find a _lower bound_ on the optimization objective.  Because (by definition) _any_ feasible $\delta$ will give us a lower bound, this is equivalent to just "trying to empirically solve the optimization problem", i.e., "find an adversarial example."  This is by far the most common strategy for solving the problem (motivated largely by the fact that for neural networks in general, problems of local optima don't seem as bad as initially may be thought). However, in order to both find strong adversarial examples _and_ to use this approach to train robust classifiers, it is important that we solve this problem _well_, which we will discuss shortly.
2. We can attempt to solve the optimization problem _exactly_.  This is going to be a challenging problem, of course, but for many networks activations functions we can formulation the exact maximization problem as a _combinatorial_ optimization problem, and solve it exactly using techniques such as mixed integer programming.  These methods will of course have substantial challenges in scaling to large models, but for small problems they highlight an important point, that it _is_ possible to construct exact solutions to the optimization problem in some cases.
3. Finally, we can also _upper bound_ the optimization objective. The basic strategy here will be consider a _relaxation_ of the network structure, such that this relaxed version contains the original network, but is built in a manner that is much easier to optimize exactly over. This will involve building _convex relaxations_ of the network structure itself.  These methods are a bit different in that they do _not_ typically construct an actual adversarial example for the real network (because they operate over a relaxed model which is not equivalent to the original one), but they can produce certifications that a network is _provably_ robust against an adversarial attack.  Futher, when combined with robust optimization for training, these methods make up the state of the art in training provably robust models.

Each of these strategies is worth considering in some detail, and this will make up the bulk of this chapter.

### Some example networks

Before we move to discussing the actual attacks, let's train a few simple networks on the MNIST problem.  Again, as we said in the previous Chapter, MNIST is of course an imperfect testbed for such settings (because some "simple" strategies like discretization work too well", but it still captures many of the fundamental principles of adversarial robustness.  We're going to actually consider three different architectures here: a two-layer or four-layer fully-connected network, and a network with four convolutional layers plus one fully-connected layer.

In [ ]:
use_gpu = False
use_saved_nets = True

In [ ]:
if use_gpu:
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
else:
    device = "cpu"

In [ ]:
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.shape[0], -1)    

model_dnn_2 = nn.Sequential(Flatten(), nn.Linear(784,200), nn.ReLU(), 
                            nn.Linear(200,10)).to(device)

model_dnn_4 = nn.Sequential(Flatten(), nn.Linear(784,200), nn.ReLU(), 
                            nn.Linear(200,100), nn.ReLU(),
                            nn.Linear(100,100), nn.ReLU(),
                            nn.Linear(100,10)).to(device)

model_cnn = nn.Sequential(nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(),
                          nn.Conv2d(32, 32, 3, padding=1, stride=2), nn.ReLU(),
                          nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
                          nn.Conv2d(64, 64, 3, padding=1, stride=2), nn.ReLU(),
                          Flatten(),
                          nn.Linear(7*7*64, 100), nn.ReLU(),
                          nn.Linear(100, 10)).to(device)


The following code trains the models (you can skip below to directly load pre-trained models if you don't want to go through the training).

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

mnist_train = datasets.MNIST("../data", train=True, download=True, transform=transforms.ToTensor())
mnist_test = datasets.MNIST("../data", train=False, download=True, transform=transforms.ToTensor())
train_loader = DataLoader(mnist_train, batch_size = 100, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size = 100, shuffle=False)

In [ ]:
def epoch(loader, model, opt=None):
    total_loss, total_err = 0.,0.
    for X,y in loader:
        X,y = X.to(device), y.to(device)
        yp = model(X)
        loss = nn.CrossEntropyLoss()(yp,y)
        if opt:
            opt.zero_grad()
            loss.backward()
            opt.step()
        
        total_err += (yp.max(dim=1)[1] != y).sum().item()
        total_loss += loss.item() * X.shape[0]
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)

In [ ]:
if not use_saved_nets:
    opt = optim.SGD(model_dnn_2.parameters(), lr=1e-1)
    for _ in range(10):
        train_err, train_loss = epoch(train_loader, model_dnn_2, opt)
        test_err, test_loss = epoch(test_loader, model_dnn_2)
        print(*("{:.6f}".format(i) for i in (train_err, train_loss, test_err, test_loss)), sep="\t")

In [ ]:
if not use_saved_nets:
    opt = optim.SGD(model_dnn_4.parameters(), lr=1e-1)
    for _ in range(10):
        train_err, train_loss = epoch(train_loader, model_dnn_4, opt)
        test_err, test_loss = epoch(test_loader, model_dnn_4)
        print(*("{:.6f}".format(i) for i in (train_err, train_loss, test_err, test_loss)), sep="\t")

In [ ]:
if not use_saved_nets:
    opt = optim.SGD(model_cnn.parameters(), lr=1e-1)
    for t in range(10):
        train_err, train_loss = epoch(train_loader, model_cnn, opt)
        test_err, test_loss = epoch(test_loader, model_cnn)
        if t == 4:
            for param_group in opt.param_groups:
                param_group["lr"] = 1e-2
        print(*("{:.6f}".format(i) for i in (train_err, train_loss, test_err, test_loss)), sep="\t")

In [ ]:
if not use_saved_nets and not use_gpu:
    torch.save(model_dnn_2.state_dict(), "model_dnn_2_cpu.pt")
    torch.save(model_dnn_4.state_dict(), "model_dnn_4_cpu.pt")
    torch.save(model_cnn.state_dict(), "model_cnn_cpu.pt")
    
if not use_saved_nets and use_gpu:
    torch.save(model_dnn_2.state_dict(), "model_dnn_2_gpu.pt")
    torch.save(model_dnn_4.state_dict(), "model_dnn_4_gpu.pt")
    torch.save(model_cnn.state_dict(), "model_cnn_gpu.pt")

If you don't want to run all the code above, you can simply load the models we trained using the following commands.

In [ ]:
if use_saved_nets and not use_gpu:
    model_dnn_2.load_state_dict(torch.load("model_dnn_2_cpu.pt"))
    model_dnn_4.load_state_dict(torch.load("model_dnn_4_cpu.pt"))
    model_cnn.load_state_dict(torch.load("model_cnn_cpu.pt"))
    
if use_saved_nets and use_gpu:
    model_dnn_2.load_state_dict(torch.load("model_dnn_2_gpu.pt"))
    model_dnn_4.load_state_dict(torch.load("model_dnn_4_gpu.pt"))
    model_cnn.load_state_dict(torch.load("model_cnn_gpu.pt"))

Together, these models will serve as a testbed for investigating the ability of different methods for solving the maximization problem that we care about.

## Lower bounding the optimization problem: adversarial attacks

Let's start by considering perhaps the simplest way of solving the optimization problem we care about, the task of maximizing
\begin{equation}
\DeclareMathOperator*{\maximize}{maximize}
\maximize_{\|\delta\| \leq \epsilon} \ell(h_\theta(x + \delta), y).
\end{equation}
In fact we already saw one example for how to do this in our introductory lecture, but the basic idea here is quite simple: using backpropagation, we can compute the gradient of the loss function with respect to the perturbation $\delta$ itself, so let's just perform gradient descent on $\delta$ to maximize our objective.  However, we also need to ensure that $\delta$ stays within the norm bound $\epsilon$, so after each step, we can project back into this space.



### The Fast Gradient Sign Method (FGSM): A Single-Step Variant of PGD

Let's consider in a bit more detail how we might do the attack we mentioned above.  For some given example $x$, we know that we are going to adjust $\delta$ in the direction of it's gradient, i.e., we will first compute the gradient
\begin{equation}
g := \nabla_\delta \ell(h_\theta(x + \delta),y)
\end{equation}
using simple backpropagation; note that if we're evaluating this gradient at $\delta=0$ (as we would at the first timestep), then this is also just equal to $\nabla_x \ell(h_\theta(x),y)$, but we'll express the gradients in terms of $\delta$ to be consistent with how we do things going forward.

In order to maximize loss, we want to adjust delta in the direction of this gradient, i.e., take a step
\begin{equation}
\delta := \delta + \alpha g
\end{equation}
for some step size $\alpha$ and then project back into the norm ball defined by $\|\delta\| \leq \epsilon$.  But how big a step do we take?  For concreteness, let's consider the particular case of the $\ell_\infty$ norm $\|\delta\|_\infty \leq \epsilon$, where, as we mentioned before, projecting onto this norm ball simply involves clipping values of $\delta$ to lie within the range $[-\epsilon, \epsilon]$.  If our initial $\delta$ is zero, this gives the update
\begin{equation}
\delta := \mathrm{clip}(\alpha g, [-\epsilon, \epsilon]).
\end{equation}
Now, how big of a step size should we take?  If we want to make increase the loss as much as possible, it makes sense to take as large a step as possible, i.e., take $\alpha$ to be very large (of course knowing that we won't take _that_ big a step, since we're projecting back into the $\ell_\infty$ ball afterwards).  It is not hard to see that for $\alpha$ large enough, the relative sizes of the entries of $g$ won't matter: we will simply take $\delta_i$ to be either $+\epsilon$ or $-\epsilon$ depending upon the sign of $g_i$.  In other words, for large $\alpha$, this update becomes
\begin{equation}
\delta := \epsilon \cdot \mathrm{sign}(g).
\end{equation}

This is know as the Fast Gradient Sign Method (FGSM) [\cite{Goodfellow2014}], and it was one of the first methods for constructing adversarial examples proposed by the deep learning community.

Let's see what this looks like in PyTorch.

In [ ]:
def fgsm(model, X, y, epsilon):
    """Construct FGSM adversarial examples on the examples X
    
    Pytorch hints: use `grad.detach()` to use the gradient value itself.
    """
    delta = torch.zeros_like(X, requires_grad=True)
    loss = nn.CrossEntropyLoss()(model(X + delta), y)
    loss.backward()
    return epsilon * delta.grad.detach().sign()

Let's look at a few of the examples generated by the method, what the classifier predicts the origina images to be, and what the classifier predicts the modifed images to be.

In [ ]:
for X,y in test_loader:
    X,y = X.to(device), y.to(device)
    break
    
def plot_images(X,y,yp,M,N):
    f,ax = plt.subplots(M,N, sharex=True, sharey=True, figsize=(N,M*1.3))
    for i in range(M):
        for j in range(N):
            ax[i][j].imshow(1-X[i*N+j][0].cpu().numpy(), cmap="gray")
            title = ax[i][j].set_title("Pred: {}".format(yp[i*N+j].max(dim=0)[1]))
            plt.setp(title, color=('g' if yp[i*N+j].max(dim=0)[1] == y[i*N+j] else 'r'))
            ax[i][j].set_axis_off()
    plt.tight_layout()

In [ ]:
### Illustrate original predictions
yp = model_dnn_2(X)
plot_images(X, y, yp, 3, 6)

In [ ]:
### Illustrate attacked images
delta = fgsm(model_dnn_2, X, y, 0.1)
yp = model_dnn_2(X + delta)
plot_images(X+delta, y, yp, 3, 6)

With relatively minor changes to the images (certainly none which would change our own prediction of the image label), we go from one incorrect prediction, to one correct prediction for this entire set we consider.  It's worth pointing out here that fully connected networks are particularly succeptible to these issues, and ConvNets slightly less so (but of course they are still very sensitive).  Let's see what the same attack looks like for our CNN model.

In [ ]:
### Illustrate attacked images
delta = fgsm(model_cnn, X, y, 0.1)
yp = model_cnn(X + delta)
plot_images(X+delta, y, yp, 3, 6)

Seeing the examples visually is nice, but let's evaluate the performance of the attack methods a bit more rigorously.  What do the test errors of our classifiers look like when evaluating against against the FGSM adversarial attack?

In [ ]:
def epoch_adversarial(model, loader, attack, *args):
    """
    Find adversarial examples for each example in the dataset.
    """
    total_loss, total_err = 0.,0.
    for X,y in loader:
        X,y = X.to(device), y.to(device)
        delta = attack(model, X, y, *args)
        yp = model(X + delta)
        loss = nn.CrossEntropyLoss()(yp,y)
        
        total_err += (yp.max(dim=1)[1] != y).sum().item()
        total_loss += loss.item() * X.shape[0]
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)

In [ ]:
print("2-layer DNN:", epoch_adversarial(model_dnn_2, test_loader, fgsm, 0.1)[0])
print("4-layer DNN:", epoch_adversarial(model_dnn_4, test_loader, fgsm, 0.1)[0])
print("        CNN:", epoch_adversarial(model_cnn, test_loader, fgsm, 0.1)[0])

Before we move on, there are a few important points to be made about FGSM.  First, it's important to emphasize that FGSM is specifically an attack under an $\ell_\infty$ norm bound: FGSM _is_ just a single projected gradient descent step under the $\ell_\infty$ constraint.  Thus, we need to consider and evaluate FGSM in the context of other $\ell_\infty$ attacks.  But it is also very easy to generalize this procedure (and others) to other norms, a topic we will discuss in some detail below.

The second point to make is that, if you recall the methods we presented in the previous chapter, it should be apparent that FGSM is _exactly_ the optimal attack against a _linear_ binary classification model under the $\ell_\infty$ norm.  This hopefully gives some additional helpful understanding of what FGSM is doing: it assumes that the linear approximation of the hypothesis given by its gradient at the point $x$ is a reasonably good approximation to the function over the entire region $\|\delta\|_\infty \leq \epsilon$.  It also, however, hints right away at the potential _disadvantages_ to the FGSM attack: because we know that neural networks are _not_ in fact linear even over a relatively small region, if we want a stronger attack we likely want to consider better methods at maximizing the loss function than a single projected gradient step.

### Projected gradient descent

This discussion immediately brings us to the next approach we would want to consider for solving the  problem, just doing projected gradient descent (i.e., the above procedure, but iterating it and with a smaller step size).  [Note: technically speaking, this is gradient _ascent_ since we are maximizing a function rather than minimization, but it's common to just refer to the process as gradient descent.] . This was also called the "basic iterative procedure" in it's first appearance within the adversarial attacks literature, but essentially it is just the obvious choice for how we might maximize the  objective a bit more carefully than with FGSM.  To re-iterate the basic approach (it is actually described directly above in the discussion of FGSM, and it is also the approach we used in the introductory section), the basic PGD algorithm simply iterates the updates.
\begin{equation}
\begin{split}
& \mbox{Repeat:} \\
& \quad \delta := \mathcal{P}(\delta + \alpha \textsf{sign}(\nabla_\delta \ell(h_\theta(x+\delta), y)))
\end{split}
\end{equation}
where $\mathcal{P}$ denotes the projection onto the ball of interest (for example, clipping in the case of the $\ell_\infty$ norm).  Of course, with PGD we now have more choices we need to make when specifying the attack, such as the actual stepsize itself, and the number of iterations (we'll discuss some rules of thumb shortly once we present a slightly modified version of the above iteration).  We can implement this attack in the following manner, where we're here going to implement the gradient descent procedure rather than rely on one of PyTorch's optimizers, as we want to see what's going on a bit more explicitly (PyTorch's SGD also includes terms like momentum, which actually usually are able to optimize the term _better_, but we want to have as little black-box procedures here as possible).

In [ ]:
def pgd_linf(model, X, y, epsilon, alpha, num_iter):
    """Construct PGD adversarial examples on the examples X
    
    Pytorch hints: 
    Reset gradients with grad.zero_() at the end of each iteration
    """
    delta = torch.zeros_like(X, requires_grad=True)
    for t in range(num_iter):
        loss = nn.CrossEntropyLoss()(model(X + delta), y)
        loss.backward()
        delta.data = (delta + alpha*delta.grad.detach().sign()).clamp(-epsilon,epsilon)
        delta.grad.zero_()
    return delta.detach()

In [ ]:
### Illustrate attacked images
delta = pgd_linf(model_cnn, X, y, epsilon=0.1, alpha=1e-2, num_iter=40)
yp = model_cnn(X + delta)
plot_images(X+delta, y, yp, 3, 6)

We're still not creating adversarial example for all cases, but we're doing a lot better than the FGSM algorithm.  It's also quite a bit easier to choose step sizes than before.  Since the step size $\alpha$ is on the same scale as the total perturbation bound $\epsilon$, it makes sense to choose $\alpha$ to be some reasonably small fraction of $\epsilon$, and then choose the number of iterations to be a small multiple of $\epsilon/\alpha$.  Let's see how these perform as an attack over the whole test set.

In [ ]:
print("2-layer DNN:", epoch_adversarial(model_dnn_2, test_loader, pgd_linf, 0.1, 1e-2, 40)[0])
print("4-layer DNN:", epoch_adversarial(model_dnn_4, test_loader, pgd_linf, 0.1, 1e-2, 40)[0])
print("CNN:", epoch_adversarial(model_cnn, test_loader, pgd_linf, 0.1, 1e-2, 40)[0])

That is a reasonable improvement over the FGSM attack.  There is one last strategy we can perform to improve performance even slightly more: randomization; this technique is not that often used in practice as it is relatively cost, but it highlights an important point.

The performance of PGD is still limited by the possibility of local optima within the objective.  And while it's not possible to avoid the reality of local optima entirely, we can mitigate the problem slightly by random restarts.  In other words, we don't just run PGD once, but we run it multiple times from different random locations within the $\ell_\infty$ ball of choice.  This gives a minor but not completely negligible improvement, but importantly highlights the fact that there _are_ local optima that PGD started at the zero point will find, and which can be avoided to some extent just by randomization.  On the downside, the approach as written naively below also increases the runtime by a factor equal to the number of restarts, so may not be practical in many settings.

In [ ]:
def pgd_linf_rand(model, X, y, epsilon, alpha, num_iter, restarts):
    """ Construct PGD adversarial examples on the samples X, with random restarts
    
    For the final delta, take the maximum over all restarts.
    """
    max_loss = torch.zeros(y.shape[0]).to(y.device)
    max_delta = torch.zeros_like(X)
    
    for i in range(restarts):
        delta = torch.rand_like(X, requires_grad=True)
        delta.data = delta.data * 2 * epsilon - epsilon
        
        for t in range(num_iter):
            loss = nn.CrossEntropyLoss()(model(X + delta), y)
            loss.backward()
            delta.data = (delta + alpha*delta.grad.detach().sign()).clamp(-epsilon,epsilon)
            delta.grad.zero_()
        
        all_loss = nn.CrossEntropyLoss(reduction='none')(model(X+delta),y)
        max_delta[all_loss >= max_loss] = delta.detach()[all_loss >= max_loss]
        max_loss = torch.max(max_loss, all_loss)
        
    return max_delta

In [ ]:
print("CNN:", epoch_adversarial(model_cnn, test_loader, pgd_linf_rand, 0.1, 1e-2, 40, 10)[0])

### Targeted attacks

What we have considered so far are "untargeted" attacks, meaning they effectively try to change the label to _any_ alternative, rather than change it to a particular alternative.  As a different task, which we saw briefly in the introduction, we can change the attack to try to convert the prediction to a particular alernative.  This is a task known as a "targetted attack", and it can be achieved using the same strategy overall strategy as we did previously.  However, in this case the only difference is that instead of maximizing the loss of the true label, we maximize the loss of the loss of the true label and also minimize the loss for the alternative label.  This is equivalent to solving the optimization problem
\begin{equation}
\maximize_{\|\delta\| \leq \epsilon} \left ( \ell(h_\theta(x + \delta), y) - \ell(h_\theta(x + \delta), y_{\mathrm{targ}}) \right ) \equiv \maximize_{\|\delta\| \leq \epsilon} \left ( h_\theta(x + \delta)_{y_{\mathrm{targ}}} - h_\theta(x + \delta)_y \right )
\end{equation}
Let's see what this looks like, using a PGD attack (without randomized restarts).  Note that in order to achieve our targetted class in most of these cases on MNIST, we use a slightly larger perturbation region, $\epsilon=0.2$.

In [ ]:
def pgd_linf_targ(model, X, y, epsilon, alpha, num_iter, y_targ):
    """ Construct targeted adversarial examples on the examples X"""
    delta = torch.zeros_like(X, requires_grad=True)
    for t in range(num_iter):
        yp = model(X + delta)
        loss = (yp[:,y_targ] - yp.gather(1,y[:,None])[:,0]).sum()
        loss.backward()
        delta.data = (delta + alpha*delta.grad.detach().sign()).clamp(-epsilon,epsilon)
        delta.grad.zero_()
    return delta.detach()

Let's look at trying to make the class label all equation to 2.

In [ ]:
delta = pgd_linf_targ(model_cnn, X, y, epsilon=0.2, alpha=1e-2, num_iter=40, y_targ=2)
yp = model_cnn(X + delta)
plot_images(X+delta, y, yp, 3, 6)

This looks pretty good: albeit with a slightly larger $\epsilon$, we can fool the classifier into predicting that all the examples are class 2 (note that the actual 2 is unchanged, because the loss function in this case is always exactly zero).  Let's try using a target class of 0 instead.

In [ ]:
delta = pgd_linf_targ(model_cnn, X, y, epsilon=0.2, alpha=1e-2, num_iter=40, y_targ=0)
yp = model_cnn(X + delta)
plot_images(X+delta, y, yp, 3, 6)

While we are able to fool the classifier for all the non-zero digits, it's worth pointing out that we don't actually achieve the target class here in all cases.  This is because the optimization objective we are maximizing is the class logit for the zero minus the class logit for the true class.  But we don't actually care what happens to the other classes, and in some cases, the best way to make the class 0 logit high is to make another class logit even higher.  We can get around this by modifying our objective to maximize the target class logit and minimize _all_ the other logits, i.e.,
\begin{equation}
\maximize_{\|\delta\| \leq \epsilon} \left ( h_\theta(x + \delta)_{y_{\mathrm{targ}}} - \sum_{y' \neq y_{\mathrm{targ}}} h_\theta(x + \delta)_{y'} \right )
\end{equation}

In [ ]:
def pgd_linf_targ2(model, X, y, epsilon, alpha, num_iter, y_targ):
    """Construct targeted adversarial examples on the examples X"""
    delta = torch.zeros_like(X, requires_grad=True)
    for t in range(num_iter):
        yp = model(X + delta)
        loss = 2*yp[:,y_targ].sum() - yp.sum()
        loss.backward()
        delta.data = (delta + alpha*delta.grad.detach().sign()).clamp(-epsilon,epsilon)
        delta.grad.zero_()
    return delta.detach()

In [ ]:
delta = pgd_linf_targ(model_cnn, X, y, epsilon=0.2, alpha=1e-2, num_iter=40, y_targ=0)
yp = model_cnn(X + delta)
plot_images(X+delta, y, yp, 3, 6)

This is a more difficult objective than the previous one, so we aren't able to fool the classifier as much.  But when we _do_ fool the classifier, it more consistently (even if still not perfectly) able to predict the target class.

### What about all those other attacks you've read about?

As a final note, we should emphasize that the presentation we present here is different from the "standard" set of attacks that are often presented in many papers.  In most adversarial example papers, you will likely see mention of attacks such as FGSM, but also CW [\cite{}], DeepFool [\cite{}], and many others.  While mentioning these attack names is useful for historical purposes (the associated papers were often the first two present and empirically study the methods), at this point our belief is that it is more useful to refer to attacks more formally by 1) the norm ball perturbation they consider, and 2) the method they use for optimizing over that norm ball.  All attacks in the literature are strategies that involve these two, and PGD methods are empirically the most effective manner for optimizing over virtually any norm ball.  But (and again, this is now our, Zico and Aleksander's, opinion) referring to attacks by the named method for solving them obscures the fundamental similarity between all the different approaches.  At the very least, we hope that we attack methods can be very explicit about the nature of the precise norm ball and optimization strategy they are using, rather than presenting it in terms of the algorithmic steps alone.